<a href="https://colab.research.google.com/github/saadatialirezam-create/bioinformatics-project/blob/main/occlusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
with open('/content/drive/MyDrive/brain_tumor_lists_occlusion.pkl', 'rb') as f:
    data = pickle.load(f)

notumor_imgs_occlusion_2x = data['notumor']
pituitary_imgs_occlusion_2x = data['pituitary']
meningioma_imgs_occlusion_2x = data['meningioma']
glioma_imgs_occlusion_2x = data['glioma']

In [3]:
import gc
import numpy as np
n_notumor    = len(notumor_imgs_occlusion_2x)
n_pituitary  = len(pituitary_imgs_occlusion_2x)
n_meningioma = len(meningioma_imgs_occlusion_2x)
n_glioma     = len(glioma_imgs_occlusion_2x)

total_imgs = []
labels = []

total_imgs.extend(notumor_imgs_occlusion_2x)
labels.extend([0] * n_notumor)

total_imgs.extend(pituitary_imgs_occlusion_2x)
labels.extend([1] * n_pituitary)

total_imgs.extend(meningioma_imgs_occlusion_2x)
labels.extend([2] * n_meningioma)

total_imgs.extend(glioma_imgs_occlusion_2x)
labels.extend([3] * n_glioma)

labels = np.array(labels, dtype=np.int64)

print(len(total_imgs), labels.shape)

del notumor_imgs_occlusion_2x
del meningioma_imgs_occlusion_2x
del pituitary_imgs_occlusion_2x
del glioma_imgs_occlusion_2x
gc.collect()

11424 (11424,)


188

In [4]:
total_imgs = np.asarray(total_imgs, dtype=np.uint8)
y = np.asarray(labels, dtype=np.int32)

In [5]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(
    total_imgs, y,
    test_size=0.2,
    random_state=494,
    stratify=y
)

print("train_x:", train_x.shape, train_x.dtype)
print("test_x:", test_x.shape, test_x.dtype)

train_x: (9139, 224, 224, 3) uint8
test_x: (2285, 224, 224, 3) uint8


In [6]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 4
IMG_SIZE = 224

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = layers.Rescaling(1./255)(inputs)

outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"]
)

model.summary()

hist = model.fit(
    train_x, train_y,
    validation_data=(test_x, test_y),
    epochs=30,
    batch_size=32,
    verbose=2
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 4)              │     4,054,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,054,695 (15.47 MB)

 Trainable params: 4,012,672 (15.31 MB)

 Non-trainable params: 42,023 (164.16 KB)

KeyboardInterrupt: 